＃価格は正しいです

今日、商品の価格を推定するためのより複雑なソリューションを構築しています。

1.このノートブック：400,000のトレーニングデータを使用してRAGデータベースを作成します
2。Day2.1ノートブック：2dで視覚化します
3。2.2日目ノートブック：3Dで視覚化します
4。Day2.3ノートブック：GPT-4O-MINIでRAGパイプラインを構築してテストする
5。Day2.4ノートブック：（a）ランダムフォレストプライザー（b）すべての価格からの寄付を可能にするアンサンブルプライザーを作成する

うーん！それは一日で乗り越えるのにたくさんのことです！

＃＃ ご注意ください：

私たちはすでに、独自のファインチューニングされたLLMを使用して非常に強力な製品推定器を持っています。ほとんどの人はそれに非常に満足するでしょう！これらの追加の手順を追加する主な理由は、RAGとエージェントワークフローで専門知識を深めることです。


In [ ]:
# 輸入

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [ ]:
# Hugging Faceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# 顔を抱き締めるためにログインした後の別の輸入品 - ありがとう、Trung N.！

from items import Item

## PKLファイルに戻ります

6週目にデータキュレーションを楽しんだように、おそらくそのプロセス全体を再び経験したくないでしょう！

次に作成したPKLファイルを再利用しましょう。ファイル「train.pkl」と「test.pkl」を今週の8フォルダーにコピーするか、ここからダウンロードすることもできます。

https://drive.google.com/drive/folders/1f_izgybvs9o0j5sb3xmtteqb3bxllzrw?usp=drive_link

In [ ]:
# このフォルダーにtrain.pklを使用すると、これを実行できます。

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [ ]:
train[0].prompt

# 次に、Chroma DataStoreを作成します

5週目に、架空の会社Insurellmのオブジェクトの塊を表す123のドキュメントを含むChroma Datastoreを作成しました。

次に、トレーニングデータセットから400,000の製品を備えたChroma Datastoreを作成します！それは本物になっています！

Langchainは使用していませんが、APIは非常に簡単で、以前と一致していることに注意してください。

特別なメモ：Chromaがクラッシュし、Windowsユーザーの場合は、以前のバージョンのChromaライブラリに戻って次を試してみてください。  
`！pipインストールChromadb == 0.5.0`  
学生のKelly Zに感謝します。これを見つけて、github Issue [here](https://github.com/chroma-core/chroma/issues/2513)を指摘してくれました。 
ただし、クロマを戻さずに最初に試してください。

In [ ]:
client = chromadb.PersistentClient(path=DB)

In [ ]:
# コレクションが存在するかどうかを確認し、それが行われている場合は削除します
collection_name = "products"

# 古いバージョンのChromaの場合、後続のラインの代わりにこの行を使用します
# 既存の_collection_names = [collection.name for collection in client.list_collections（）]
existing_collection_names = client.list_collections()

if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

＃sentenceTransfomerの紹介

All-Minilmは、文と段落を384次元密度のベクトル空間にマッピングするHugging Faceの非常に便利なモデルであり、セマンティック検索などのタスクに最適です。

https://huggingface.co/sentence-transformers/all-minilm-l6-v2

ローカルでかなり迅速に実行できます。

前回、Openai埋め込みを使用してベクトル埋め込みを生成しました。 Openai Embeddingsと比較した利点：
1。それは無料で速いです！
3.ローカルで実行できるので、データが私たちの箱を離れることはありません - あなたが個人的なぼろきれを構築している場合に役立つかもしれません


In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# テキストのリストを渡し、ベクターの数字を取り戻す

vector = model.encode(["Well hi there"])[0]
vector

In [ ]:
# クイックサイドバー - ビデオへの追加 - ベクターを比較する機能

import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [ ]:
# そして、コンテキストにいくつかの単語を追加することで物事を変える方法を見てみましょう！

how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

In [ ]:
# メインストーリーに戻りましょう - 私たちがベクトル化できるものを作ってみましょう

def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [ ]:
description(train[0])

## 今、私たちはぼろきれデータストアに入力します

次のセルは、Chromaの400,000のアイテムに浸透します。

これに時間がかかりすぎる場合は、ドキュメントの数をお気軽に削減してください！あなたは次のように変更できます：  
`number_of_documents = 20000`  
そして、それは完全に良いぼろきれのパイプラインのためにたくさんです。

実行中に以下のセルを中断した場合、再び実行する前に、Chroma DataStore（コレクションを削除する以前のセルを再実行することにより）をクリアする必要があるかもしれないことに注意してください。それ以外の場合は、同じIDを持つ既存のドキュメントがあることに不満を言います。

In [ ]:
NUMBER_OF_DOCUMENTS = len(train)

# 400,000全員を待たないであれば解除
# number_of_documents = 20000

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )